# Lab 11: Integer Linear programming and Dinamic Programming 
In this lab, we will see Integer Linear programming and Dynamic programming (DP). 

For the Integer Linear Programming, you will proceed as in the previous lab, writing some models and solving the problem. For Dynamic Programming, we will ask you to solve the knapsack problem with DP. 

In [ ]:
# !pip install -q condacolab
# import condacolab
# 
# condacolab.install()
# !conda install pyscipopt

In [ ]:
# !pip install scikit-optimize
# !pip install treed

In [ ]:
import pyscipopt
from pyscipopt import Model, quicksum
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE
import math
from treed import TreeD


class LPstatEventhdlr(Eventhdlr):
    """PySCIPOpt Event handler to collect data on LP events."""

    transvars = {}

    def collectNodeInfo(self, firstlp=True):
        objval = self.model.getSolObjVal(None)

        LPsol = {}
        if self.transvars == {}:
            self.transvars = self.model.getVars(transformed=True)
        for var in self.transvars:
            solval = self.model.getSolVal(None, var)
            LPsol[var.name] = self.model.getSolVal(None, var)

        # skip duplicate nodes
        # if self.nodelist and LPsol == self.nodelist[-1].get("LPsol"):
        #     return
        node = self.model.getCurrentNode()
        if node.getNumber() != 1:
            parentnode = node.getParent()
            parent = parentnode.getNumber()
        else:
            parent = 1
        depth = node.getDepth()
        age = self.model.getNNodes()
        condition = math.log10(self.model.getCondition())
        iters = self.model.lpiGetIterations()
        pb = self.model.getPrimalbound()
        if pb >= self.model.infinity():
            pb = None

        nodedict = {
            "number": node.getNumber(),
            "LPsol": LPsol,
            "objval": objval,
            "parent": parent,
            "age": age,
            "depth": depth,
            "first": firstlp,
            "condition": condition,
            "iterations": iters,
            # "variables": self.model.getNVars(),
            # "constraints": self.model.getNConss(),
            "rows": self.model.getNLPRows(),
            "primalbound": pb,
            "dualbound": self.model.getDualbound(),
            "time": self.model.getSolvingTime()
        }

        self.nodelist.append(nodedict)

    def eventexec(self, event):

        if event.getType() == SCIP_EVENTTYPE.FIRSTLPSOLVED:
            self.collectNodeInfo(firstlp=True)
        elif event.getType() == SCIP_EVENTTYPE.LPSOLVED:
            self.collectNodeInfo(firstlp=False)
        else:
            print("unexpected event:" + str(event))
        return {}

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.LPEVENT, self)


def convertSolToDict(sol):
    ssol = str(sol)
    items = ssol[1:-1].split(",")
    res = {}
    for item in items:
        k = str(item.split(":")[0])
        v = float(item.split(":")[1])
        res[k] = v
    
    return res


def solve_model(model):
    model.setPresolve(pyscipopt.SCIP_PARAMSETTING.OFF)
    model.setHeuristics(pyscipopt.SCIP_PARAMSETTING.OFF)
    model.disablePropagation()
    #model.redirectOutput()
    nodelist = []
    eventhdlr = LPstatEventhdlr()
    eventhdlr.nodelist = nodelist
    model.includeEventhdlr(
        eventhdlr, "LPstat", "generate LP statistics after every LP event"
    )
    model.optimize()
    frontier_history = []
    for nd in nodelist:
        frontier_history.append((convertSolToDict(nd['LPsol']), nd['objval']))
    best_sol = model.getBestSol()
    best_val = model.getObjVal()
    return convertSolToDict(best_sol), best_val, frontier_history

# Ex. 0

This is an example to show how the Library works

---

Slack form:

minimize  $2x_1 + x_2 − 2x_3$

subject to 

- $0.7x_1 + 0.5x_2 +x_3 \geq 1.8$
- $x_i \in [0,1]\ \forall i$

In [ ]:
model = Model("")
v = [model.addVar(vtype='i', name=str(i), lb=0, ub=1) for i in range(3)]
c = [0.7, 0.5, 1]
model.addCons(quicksum(c[i] * v[i] for i in range(3)) >= 1.8)
sum([i for i in range(3)])
model.setObjective(2 * v[0] + v[1] - 2 * v[2], "minimize")
best_sol, best_val, partial_frontier_history = solve_model(model)

print(best_sol)
print(best_val)
print(partial_frontier_history)

# Ex. The N-queens Problem | ILP

The N queens puzzle is the problem of placing $N$ chess queens on an $N \times N (N \geq 4)$ chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. Try different values of $N$ and shows how the problem complexity increases.

---

Foreach cell in the chessboard a variable

Each cell can be either 0 or 1, where
- 0 means the cell is empty
- 1 means the cell contains a queen

We can encode the constrains on rows, columns and diagonals as a disequation, where the sum of the corresponding cells must be less or equal to 1

less or equal because if it is true that there must be a queen foreach row and column, the same does not apply for the diagonals

The problem becomes a maximization problem on the sum of every cells


In [ ]:
import numpy as np

n = 10

chessboard = np.asarray([
    np.asarray([ 
        f'({r} {c})'
        for c
        in range(n)
        ])
    for r
    in range(n)
    ])

print(chessboard)

print('-' * 90)

for r in range(0, n):
  print(chessboard[r])

print('-' * 90)

for c in range(0, n):
  print(chessboard[:, c])

print('-' * 90)

for d in range(-n + 1, n):
  print(chessboard.diagonal(d))

print('-' * 90)

for d in range(-n + 1, n):
  print(np.fliplr(chessboard).diagonal(d))

print('-' * 90)

print(np.reshape(chessboard, -1)) 

In [ ]:
n = 10

model = Model('')
chessboard = np.asarray([
    np.asarray([
        model.addVar(vtype='i', name=f'({r} {c})', lb=0, ub=1) for c in range(n)
        ])
    for r
    in range(n)
    ])

print(chessboard)

# rows
for r in range(n):
  model.addCons(quicksum(cell for cell in chessboard[r]) <= 1)

# columns
for c in range(n):
  model.addCons(quicksum(cell for cell in chessboard[c]) <= 1)

# diagonals top left - bottom right
for d in range(-n + 1, n):
  model.addCons(quicksum(cell for cell in chessboard.diagonal(d)) <= 1)


# diagonals top right - bottom left
for d in range(-n + 1, n):
  model.addCons(quicksum(cell for cell in np.fliplr(chessboard).diagonal(d)) <= 1)

model.setObjective(quicksum(cell for cell in np.reshape(chessboard, -1)), "maximize")
best_sol, best_val, partial_frontier_history = solve_model(model)

print(best_sol)
print(best_val)
print(partial_frontier_history)

# Traveling salesman problem | ILP

The goal of the TSP is to find the shortest Hamiltonian cycle (a cycle that visits each node only once) on a graph of N nodes. Solve the ILP problem and visualize a solution.

In [ ]:
# edge cost
# (a, b) cost
# find the cycle
# minimize the sum of cycle
# visit each node at most one -> each node can use at least one edge




# Ex. KNAPSACK PROBLEM | DP

Solve the Knapsack problem using DP.
You can use the Class from lab 5.

Run the solution multiple time, and change the total capacity and the number of objects and shows how the number of sub-problems changes.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt


class Knapsack_0_1:

    def __init__(self):
        self._items = [
            []
        ]
        self._BAG_CAPACITY = 10
        self.history = []
        self.values = []

    def _get_value(self, solution):
        cur_cap = self._BAG_CAPACITY
        cur_val = 0
        for i, v in enumerate(solution):
            if v == 1:
                cur_val += self._items[i]['value']
                cur_cap -= self._items[i]['volume']
            if cur_cap < 0:
                return 0
        return -cur_val

    def __call__(self, solution):
        value = self._get_value(solution)
        self.history.append(solution)
        self.values.append(value)
        return value

    def trend(self):
        plt.figure()
        plt.plot(self.values)
        plt.show()
